In [1]:
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from numba import njit,jit
import numpy.fft as fft
import scipy.stats as stats
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime
import pickle as pkl

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
zone_name = "Southern_California"
! wget "https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/Southern_California.gpx"

--2022-03-01 22:28:36--  https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/Southern_California.gpx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103498 (101K) [text/plain]
Saving to: ‘Southern_California.gpx.4’

Southern_California 100%[===================>] 101.07K  --.-KB/s    in 0.02s   

2022-03-01 22:28:36 (4.84 MB/s) - ‘Southern_California.gpx.4’ saved [103498/103498]



In [3]:
zone_name = "Southern_California"
with open(zone_name + ".gpx", 'r') as f:
    stat_xml = BeautifulSoup(f.read(), "xml")

In [4]:
station_names = []
station_pos = {}
for wpt in stat_xml.find_all("wpt"):
    name = wpt.find("name").text.lower()
    station_names.append(name)
    pos = (float(wpt.get("lat")),float(wpt.get("lon")))
    station_pos[name] = pos

In [5]:
raw_link ="https://earthquake.usgs.gov/monitoring/gps/data/networks/%s/%s/nafixed/%s.rneu"

In [6]:
#my_stats = ["coso", "towg", "worg", "whfg", "wasg", "wlhg"]

In [7]:
data = {}
for cnt,name in enumerate(station_names):
    if(cnt % 100==0):
      print(cnt/len(station_names))
    try:
        data[name] = pd.read_csv(raw_link%(zone_name, name, name), sep='\t', header=None)
        data[name].columns = columns=["DateI","DateD","N","E","U","Tech1","dN","dE","dU","Tech2","Descr",]
    except:
        print("Not found: ", name)

0.0
Not found:  azyu
0.23201856148491878
0.46403712296983757
0.6960556844547564
0.9280742459396751


In [8]:
with open("station_data", "wb") as handle:
  pkl.dump(data, handle)